## Import Libraries

In [46]:
import requests
import pandas as pd
import time
from datetime import datetime

In [47]:
# List of Selected Tickers we want to analyse 
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

In [48]:
# store stockdata here 
stock_data = []

# API URL and Key 

In [49]:
api_key = "9JPRF2OKFQM0FT0U"
base_url_template = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}'


### API Calls for Multiple Tickers
#### Storing data to Stock_Data
Transforming open, close, high, low from str to floats,
Date to date object 
volume from str to int

In [ ]:
# outputsize defaults to compact if you want 20 years of data you can change to "= full" 
# monthly data defaults to full - they returned me over 500 data points for 1 ticker 
for tick in tickers:
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={tick}&apikey={api_key}'
    try:
        r = requests.get(url)
        r.raise_for_status()  
        data = r.json()

        time_series_key = 'Time Series (Daily)' # Depends on what key is returned by the specified API function 
        if time_series_key in data:
            for date_str, values in data[time_series_key].items():
                row = {
                    'Symbol': tick,
                    'Date': date_str,  
                    'Open': float(values['1. open']),
                    'High': float(values['2. high']),
                    'Low': float(values['3. low']),
                    'Close': float(values['4. close']),
                    'Volume': int(values['5. volume'])
                }
                stock_data.append(row)
            print(f"--- Successfully collected and formatted data for {tick} ---")
            
        elif "Error Message" in data:
            print(f"API Error for {tick}: {data['Error Message']}")
        else:
            print(f"Unexpected data format for {tick}.") 
        if "Error Message" in data:
            print(f"Error for {tick}: {data['Error Message']}")

        else:
            print(f"--- Data for {tick} ---")
            print(data)
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching data for {tick}: {e}")
    time.sleep(20)    



--- Data for AAPL ---
{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'AAPL', '3. Last Refreshed': '2025-09-05', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-09-05': {'1. open': '239.9950', '2. high': '241.3200', '3. low': '238.4901', '4. close': '239.6900', '5. volume': '54870397'}, '2025-09-04': {'1. open': '238.4500', '2. high': '239.8999', '3. low': '236.7400', '4. close': '239.7800', '5. volume': '47549429'}, '2025-09-03': {'1. open': '237.2100', '2. high': '238.8500', '3. low': '234.3600', '4. close': '238.4700', '5. volume': '66061716'}, '2025-09-02': {'1. open': '229.2500', '2. high': '230.8500', '3. low': '226.9700', '4. close': '229.7200', '5. volume': '44075638'}, '2025-08-29': {'1. open': '232.5100', '2. high': '233.3800', '3. low': '231.3700', '4. close': '232.1400', '5. volume': '39418437'}, '2025-08-28': {'1. open': '230.8200', '2. high': '233.4100', '3. low': '229.3350', '4. 

Convert to Data Frame 

In [66]:
stock_df = pd.DataFrame(stock_data)

In [67]:
# convert date from str to date time
stock_df['Date'] = pd.to_datetime(stock_df['Date'])

In [68]:
stock_df.tail()

,Symbol,Date,Open,High,Low,Close,Volume
195,TSLA,2025-04-21,230.26,232.21,222.7900,227.50,97768007
196,TSLA,2025-04-17,243.47,244.34,237.6833,241.37,83404775
197,TSLA,2025-04-16,247.61,251.97,233.8900,241.55,112378737
198,TSLA,2025-04-15,249.91,258.75,247.5400,254.11,79594318
199,TSLA,2025-04-14,258.36,261.80,245.9300,252.35,100135241


## Convert to CSV

In [64]:
stock_df.to_csv('stock_data.csv', index=False)